📘 Jupyter Notebook: Visualizing and Comparing Power System Graphs with GeoPandas

In [1]:
# 📦 Section 1: Imports & Setup
from pathlib import Path
import geopandas as gpd
import pandas as pd
from grid_reducer.utils import get_ckt_from_opendss_model
from grid_reducer.network import get_graph_from_circuit
from grid_reducer.plot import graph_to_geodataframe
from grid_reducer.reducer import OpenDSSModelReducer

In [2]:
# ⚡ Section 2: Load and Parse OpenDSS Circuit

# Specify the path to your DSS master file
master_dss_path = Path("../tests/data/smartds/Master.dss")

# Load the circuit object from the OpenDSS model
ckt = get_ckt_from_opendss_model(master_dss_path)
reducer = OpenDSSModelReducer(master_dss_file=master_dss_path)
reduced_ckt1 = reducer.reduce(transform_coordinate=False, noise_level="none")
reduced_ckt2 = reducer.reduce(transform_coordinate=False, noise_level="low")
reduced_ckt3 = reducer.reduce(transform_coordinate=True, noise_level="none")
reduced_ckt4 = reducer.reduce(transform_coordinate=True, noise_level="low")

# Create a NetworkX graph from the circuit
#graph = get_graph_from_circuit(ckt)
graph1 = get_graph_from_circuit(reduced_ckt1)
graph2 = get_graph_from_circuit(reduced_ckt2)
graph3 = get_graph_from_circuit(reduced_ckt3)
graph4 = get_graph_from_circuit(reduced_ckt4)
graph5 = get_graph_from_circuit(ckt)

# Print basic info
#print(f"Number of nodes: {len(graph.nodes)}")
#print(f"Number of edges: {len(graph.edges)}")

Number of aggregated lines = 12
Number of removed lines = 24
Number of aggregated lines = 12
Number of removed lines = 24
Noise level:low; Noise scale: 0.01
Number of aggregated lines = 12
Number of removed lines = 24
Transforming coordinates...
Time: 2.3034918308258057
Number of aggregated lines = 12
Number of removed lines = 24
Transforming coordinates...
Time: 0.26082777976989746
Noise level:low; Noise scale: 0.01


In [3]:
gdf1 = graph_to_geodataframe(graph1)
gdf2 = graph_to_geodataframe(graph2)
gdf3 = graph_to_geodataframe(graph3)
gdf4 = graph_to_geodataframe(graph4)
gdf5 = graph_to_geodataframe(graph5)

In [4]:
# transform_coordinate=False, noise_level="none"
gdf1.head()

,edge_id,source,target,length_km,kv,component_type,name,geometry
0,0,0,121,0.52,7.199558,Line,line__102,"LINESTRING (-122.48087 38.2472, -122.48538 38...."
1,1,1,2,0.07,7.199558,Line,line__11,"LINESTRING (-122.51227 38.26007, -122.51293 38..."
2,2,1,3,1.97,7.199558,Line,line__128,"LINESTRING (-122.51227 38.26007, -122.5009 38...."
3,3,3,119,0.01,7.199558,Line,line__88,"LINESTRING (-122.5009 38.24649, -122.5009 38.2..."
4,4,4,123,0.00,7.199558,Line,line__44,"LINESTRING (-122.5391 38.2444, -122.53911 38.2..."


In [5]:
# transform_coordinate=False, noise_level="low"
gdf2.head()

,edge_id,source,target,length_km,kv,component_type,name,geometry
0,0,0,121,2.12,7.199558,Line,line__102,"LINESTRING (-122.47727 38.24211, -122.47366 38..."
1,1,1,2,2.31,7.199558,Line,line__11,"LINESTRING (-122.51861 38.25464, -122.49918 38..."
2,2,1,3,2.33,7.199558,Line,line__128,"LINESTRING (-122.51861 38.25464, -122.50139 38..."
3,3,3,119,0.98,7.199558,Line,line__88,"LINESTRING (-122.50139 38.24262, -122.4931 38...."
4,4,4,123,0.94,7.199558,Line,line__44,"LINESTRING (-122.53099 38.24637, -122.53114 38..."


In [6]:
# transform_coordinate=True, noise_level="none"
gdf3.head()

,edge_id,source,target,length_km,kv,component_type,name,geometry
0,0,0,121,3.60,7.199558,Line,line__102,"LINESTRING (-0.47362 0.23429, -0.46882 0.20218)"
1,1,1,2,3.59,7.199558,Line,line__11,"LINESTRING (-0.40613 0.15445, -0.40286 0.18664)"
2,2,1,3,3.58,7.199558,Line,line__128,"LINESTRING (-0.40613 0.15445, -0.4177 0.12435)"
3,3,3,119,3.46,7.199558,Line,line__88,"LINESTRING (-0.4177 0.12435, -0.44263 0.10567)"
4,4,4,123,3.68,7.199558,Line,line__44,"LINESTRING (-0.62087 -0.39656, -0.59517 -0.37564)"


In [7]:
# transform_coordinate=True, noise_level="low"
gdf4.head()

,edge_id,source,target,length_km,kv,component_type,name,geometry
0,0,0,121,6.25,7.199558,Line,line__102,"LINESTRING (-0.45265 0.23258, -0.46473 0.17759)"
1,1,1,2,2.12,7.199558,Line,line__11,"LINESTRING (-0.3937 0.15728, -0.41193 0.16295)"
2,2,1,3,3.52,7.199558,Line,line__128,"LINESTRING (-0.3937 0.15728, -0.41525 0.13398)"
3,3,3,119,3.96,7.199558,Line,line__88,"LINESTRING (-0.41525 0.13398, -0.44148 0.10976)"
4,4,4,123,4.23,7.199558,Line,line__44,"LINESTRING (-0.62556 -0.3913, -0.59345 -0.37072)"


In [8]:
# Un-reduced circuit
gdf5.head()

,edge_id,source,target,length_km,kv,component_type,name,high_kv,low_kv,kva,geometry
0,0,p12udt1266-p12uhs0_1247x,p12udt1266,0.52,7.199558,Line,l(r:p12udt1266-p12uhs0_1247)_s1,NaN,NaN,NaN,"LINESTRING (-122.48087 38.2472, -122.48538 38...."
1,1,p12udt77lv,p12ulv165,0.02,0.120089,Line,l(r:p12udt77lv-p12ulv165),NaN,NaN,NaN,"LINESTRING (-122.54487 38.20966, -122.54503 38..."
2,2,p12udt77lv,p12ulv265,0.01,0.120089,Line,l(r:p12udt77lv-p12ulv265),NaN,NaN,NaN,"LINESTRING (-122.54487 38.20966, -122.54478 38..."
3,3,p12udt77lv,p12ulv268,0.05,0.120089,Line,l(r:p12udt77lv-p12ulv268),NaN,NaN,NaN,"LINESTRING (-122.54487 38.20966, -122.54447 38..."
4,4,p12udt77lv,p12ulv309,0.02,0.120089,Line,l(r:p12udt77lv-p12ulv309),NaN,NaN,NaN,"LINESTRING (-122.54487 38.20966, -122.54497 38..."


In [9]:
# transform_coordinate=False, noise_level="none"
gdf1.explore(
    style_kwds={
        "color": "red",  # Line color
        "weight": 5,  # Line width
        "opacity": 0.8,  # Line transparency (optional)
    },
    legend=True,
)

In [10]:
# transform_coordinate=False, noise_level="low"
gdf2.explore(
    style_kwds={
        "color": "red",  # Line color
        "weight": 5,  # Line width
        "opacity": 0.8,  # Line transparency (optional)
    },
    legend=True,
)

In [11]:
# transform_coordinate=True, noise_level="none"
gdf3.explore(
    style_kwds={
        "color": "red",  # Line color
        "weight": 5,  # Line width
        "opacity": 0.8,  # Line transparency (optional)
    },
    legend=True,
)

In [12]:
# transform_coordinate=True, noise_level="low"
gdf4.explore(
    style_kwds={
        "color": "red",  # Line color
        "weight": 5,  # Line width
        "opacity": 0.8,  # Line transparency (optional)
    },
    legend=True,
)

In [13]:
# Un-reduced circuit
gdf5.explore(
    style_kwds={
        "color": "red",  # Line color
        "weight": 5,  # Line width
        "opacity": 0.8,  # Line transparency (optional)
    },
    legend=True,
)

In [14]:
# # 🔍 Section 5: Comparing Multiple Graphs Side-by-Side

# # Load a second circuit (e.g., a modified version for comparison)
# master_dss_path2 = Path("../tests/data/smartds/Master.dss")
# ckt2 = get_ckt_from_opendss_model(master_dss_path2)
# reducer = OpenDSSModelReducer(master_dss_file=master_dss_path)
# reduced_ckt = reducer.reduce()
# ckt2 = reduced_ckt
# graph2 = get_graph_from_circuit(ckt2)
# gdf2 = graph_to_geodataframe(graph2)

# # Add 'category' column to both
# gdf["source"] = "Base"
# gdf2["source"] = "Variant"

# # Merge for comparison
# combined_gdf = pd.concat([gdf, gdf2], ignore_index=True)

# # Visualize both overlaid, color-coded by source
# combined_gdf.explore(column="source", legend=True, cmap="Set2")